# Jiaming Lin

## 1

I decide to web scrape Wiki Cookbook: https://en.wikibooks.org/wiki/Cookbook:Recipes

I think it does not mention scraping, it just let me know they are not responsible for third party information and losses caused by downloading of these materials. The robots.txt file says "Crawlers that are kind enough to obey, but which we'd rather not have unless they're feeding search engines."

I will try small parts at first, and then download all materials I need once. Besides, I will wait for a second between two downloads.

## 2

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Request the web page I want to scrape
result = requests.get("https://en.wikibooks.org/wiki/Cookbook:Recipes")

In [3]:
# Get all of the content in that webpage
ws = BeautifulSoup(result.content)

In [4]:
# Find all "a" on the web page
As = ws.find_all("a")
hrefs = pd.Series([],dtype = "object")
names = pd.Series([],dtype = "object")
count = 0
# Get all names and links from all "a"
for i in As:
    hrefs[count] = i.get("href")
    names[count] = i.text
    count = count + 1 
    
# hrefs: a list of website 
# names: a list of the names of recipes
# 38 is number of first valid recipe
names = names[38:100]
hrefs = hrefs[38:100]

# recipes I scraped
print(names)

38                             Aam panna
39    Acorn Squash with Orange Marmalade
40                                 Accra
41                                 Adobo
42                         Agedashi tofu
                     ...                
95                            Bison Loaf
96              Black-Eyed Peas and Kale
97                                 Bliny
98                              Blondies
99                          BLT Sandwich
Length: 62, dtype: object


In [5]:
# Combine the names and links to a same dataframe
name_href = pd.concat([names, hrefs], axis=1)

In [6]:
# This a function to create three list
# which are a list of ingredient, repetitive names and repetitive id for a each meal
def get_ingri(href, name, theid):
    result = requests.get(href)
    # Check whether website is valid
    if result is None:
        return None, None, None
    if result.status_code != 200:
        return None, None, None
    ws = BeautifulSoup(result.content)
    # Check whether the website contains Ingredient
    if ws.find_all("span", id = "Ingredients") == []:
        return None, None, None
    else:
        # Create a list of ingredient, repetitive names and repetitive id for a each meal
        ids = []
        names = []  
        ig = []  
        ws = BeautifulSoup(result.content)
        LIS = ws.ul.find_all("li")
        for LI in LIS:
            ig.append(LI.text)
            names.append(name)
            ids.append(theid)
    
    return ig, names, ids

In [7]:
# Create a series to store ingredients
ingredients = []
recipe_name = []
ids = []

# Loop through all meals
for i in range(1,61):
    # Make a link for the web page which will be scraped
    href = "https://www.wikibooks.org/" + name_href.iloc[i,1]
    ingredient, name, theid = get_ingri(href, name_href.iloc[i,0],i)
    if ingredient != None:
        ingredients.append(ingredient)
        recipe_name.append(name)
        ids.append(theid)

In [8]:
# Get nested list out and remake a list 

# Create a list of ingredients as a column in the dataframe
ingri = []
for ig in ingredients:
    for element in ig:
        ingri.append(element)

# Create a list of names as a column in the dataframe
name = []
for same_name in recipe_name:
    for element in same_name:
        name.append(element)

# Create a list of ids as a column in the dataframe
id_list = []
for the_id in ids:
    for element in the_id:
        id_list.append(element)

In [9]:
# Split ingredients by " "
split_ig = []
for ig in ingri:
    split_ig.append(ig.split())   

# Splited ingredients
split_ig

[['1', 'medium', 'acorn', 'squash,', 'halved', 'lengthwise'],
 ['1', 'tablespoon', 'orange', 'marmalade'],
 ['1', 'teaspoon', 'butter', 'or', 'margarine'],
 ['1/8', 'teaspoon', 'salt'],
 ['2', 'cups', '(480', 'ml)', 'flour'],
 ['2', 'tsp', '(10', 'ml)', 'baking', 'powder'],
 ['1', 'tsp', '(5', 'ml)', 'sugar'],
 ['½', 'lb', '(~225', 'g)', 'chopped', 'shrimp'],
 ['2', 'spring', 'onions', 'chopped.'],
 ['¼', 'cup', '(60', 'ml)', 'green', 'pepper', 'chopped'],
 ['1', 'small', 'onions', 'chopped'],
 ['4', 'cloves', 'garlic'],
 ['1', 'tsp', '(5', 'ml)', 'thyme'],
 ['1', 'tsp', '(5', 'ml)', 'parsley'],
 ['water'],
 ['1', 'Scotch', 'bonnet', 'or', 'hot', 'pepper'],
 ['black',
  'pepper',
  'and',
  'salt',
  '(really',
  'optional',
  'here)',
  'to',
  'taste'],
 ['6-8', 'chicken', 'pieces,', 'preferably', 'legs', 'and', 'wings'],
 ['⅓',
  'cup',
  '(80',
  'mL)',
  'vinegar',
  'made',
  'from',
  'coconut',
  'juice',
  '(as',
  'a',
  'variant,',
  'try',
  'balsamic',
  'vinegar)'],
 ['¼'

In [10]:
# Create a list of text as a column in the dataframe
text = ingri.copy()
quantity = []
for ig in split_ig:
    quantity.append(ig[0])

# Create a list of unit as a column in the dataframe
unit = []
for ig in split_ig:
    str = ''.join(ig[1:2])
    unit.append(str)

# Create a list of ingredient as a column in the dataframe
real_ig = []
for ig in split_ig:
    str = ' '.join(ig[2:])
    real_ig.append(str)

In [11]:
# Make the final dataframe
recipes = pd.DataFrame(
    {"id": id_list,
     "name": name,
     "text": text,
     "quantity" :quantity,
     "unit": unit,
     "ingredient":real_ig
    })

In [12]:
# My final dataframe
recipes

,id,name,text,quantity,unit,ingredient
0,1,Acorn Squash with Orange Marmalade,"1 medium acorn squash, halved lengthwise",1,medium,"acorn squash, halved lengthwise"
1,1,Acorn Squash with Orange Marmalade,1 tablespoon orange marmalade,1,tablespoon,orange marmalade
2,1,Acorn Squash with Orange Marmalade,1 teaspoon butter or margarine,1,teaspoon,butter or margarine
3,1,Acorn Squash with Orange Marmalade,1/8 teaspoon salt,1/8,teaspoon,salt
4,2,Accra,2 cups (480 ml) flour,2,cups,(480 ml) flour
...,...,...,...,...,...,...
405,59,Bliny,5 teaspoons of butter,5,teaspoons,of butter
406,59,Bliny,1 ounce of yeast,1,ounce,of yeast
407,59,Bliny,1 egg,1,egg,
408,59,Bliny,4 cups of milk,4,cups,of milk


In [13]:
# Save my dataframe to a csv file
recipes.to_csv("recipe.csv")

## 3

In [14]:
text = pd.Series(text)
print(text[text.str.contains("salt") == True].count())
print(text[text.str.contains("sugar") == True].count())
# There are more recipes with salt

30
13


In [15]:
# I think I cannot do this problem as the unit is different, so we cannot compare the absolute quantities

## 4

In [16]:
# I spend about 8 hours